In [1]:
from langchain_openai import ChatOpenAI
_llm=ChatOpenAI(
    api_key="ollama", #ollama api_key随便写
    model="qwen2.5:7b", #看ollama中的名
    base_url="http://192.168.10.11:60006/v1",#注意后面加v1
    temperature=0.7
)

In [2]:
_llm.invoke("记住我是Skyline")

AIMessage(content='好的，我记住了，您是Skyline。有什么我可以帮助您的吗？如果您有任何问题、需要信息或想要讨论某个话题，请尽管告诉我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 33, 'total_tokens': 66, 'completion_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-458ea0a4-a3fc-405d-8c43-6dc325ffeff8-0', usage_metadata={'input_tokens': 33, 'output_tokens': 33, 'total_tokens': 66, 'input_token_details': {}, 'output_token_details': {}})

###### 大模型无法记住对话，要解决这种情况

In [3]:
_llm.invoke("我是谁？")

AIMessage(content='您是提问的用户。在我们的对话中，您可以分享任何想法或问题，我会尽我所能提供帮助或信息。如果您想了解关于自己的一些具体问题，例如您的名字、身份等，请提供更多背景信息，这样我才能更好地帮助到您。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 32, 'total_tokens': 88, 'completion_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-1719dde8-2789-4acb-a9f2-967723937780-0', usage_metadata={'input_tokens': 32, 'output_tokens': 56, 'total_tokens': 88, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
# from langchain_core.messages import HumanMessage

# store=[
#     HumanMessage("记住我是Skyline")
# ]
# rep = _llm.invoke(store)
# store.append(rep)
# store.append(HumanMessage("我是谁"))
# rep = _llm.invoke(store)
# rep

##### 创建一个管理消息历史记录

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory

_store = {}
def get_session_history(session_id):
    if session_id not in _store:
        #创建历史列表
        _store[session_id] = ChatMessageHistory()
    return _store[session_id]

In [6]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
# 定义模板
_messages = ChatPromptTemplate([
    ("system","请用中文帮助用户"),
    #这个类或函数可能用于表示消息内容的占位符。
    # 在构建聊天界面时，占位符通常用于在用户界面中预留显示消息的空间。
    # 当实际的消息数据被检索或生成后，这些占位符会被实际的消息内容替换。
    MessagesPlaceholder(variable_name="history_contents"),
    ("human","{content}")
])

In [7]:
from langchain_core.output_parsers import StrOutputParser
# 创建一个链
_chain = _messages|_llm|StrOutputParser()

##### 使用

In [8]:
from langchain_core.runnables import RunnableWithMessageHistory

_his_chain = RunnableWithMessageHistory(
    _chain,
    get_session_history,
    # 定义两个键
    input_messages_key="content",
    history_messages_key="history_contents"
)
# 配置config的格式是固定的
#设置为第一个用户
rep01 = _his_chain.invoke({"content":"记住我叫Skyline"},config={"configurable":{"session_id":"1"}}) 
print(rep01)


好的，你可以说：“你好，Skyline”，我会这样回应你。如果你需要任何帮助或者想进行对话，可以随时告诉我。


In [9]:
rep02 = _his_chain.invoke({"content":"我叫什么名字"},config={"configurable":{"session_id":"1"}})
print(rep02)

你叫Skyline。如果你有其他问题或需要帮助的事情，请随时告诉我！


##### 更换会话，则不会记住另一个会话的历史

In [10]:
rep03 = _his_chain.invoke({"content":"我叫什么名字"},config={"configurable":{"session_id":"2"}})
print(rep03)

您在对话中并没有提到您的名字，所以我无法直接得知您叫什么名字。如果您告诉我或者在后续的对话中提到您的名字，我很乐意帮您确认或使用它进行交流。
